In [ ]:
pip -q install annoy wget transformers

In [ ]:
import wget
wget.download('https://gist.githubusercontent.com/a1ip/0a5ec1b89e79b4490ef5992a80e72eeb/raw/643fb1089165676903b05852cb7d72cb2f51b2f8/rst.json')

'rst.json'

In [ ]:
# @title Сделаем красиво
import json
import pandas as pd

with open('rst.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

rows = []
for book in data['Books']:
    book_id = book['BookId']
    book_name = book['BookName']
    for chapter in book['Chapters']:
        chapter_id = chapter['ChapterId']
        for verse in chapter['Verses']:
            verse_id = verse['VerseId']
            text = verse['Text']
            rows.append((book_id, book_name, chapter_id, verse_id, text))

df = pd.DataFrame(rows, columns=['BID','Book', 'Chapter', 'Verse', 'Text'])

df

,BID,Book,Chapter,Verse,Text
0,1,Быт.,1,1,В начале сотворил Бог небо и землю.
1,1,Быт.,1,2,"Земля же была безвидна и пуста, и тьма над без..."
2,1,Быт.,1,3,И сказал Бог: да будет свет. И сталсвет.
3,1,Быт.,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
4,1,Быт.,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
31097,66,Откр.,22,17,И Дух и невеста говорят: прииди! И слышавший д...
31098,66,Откр.,22,18,И я также свидетельствую всякому слышащему сло...
31099,66,Откр.,22,19,и если кто отнимет что от слов книги пророчест...
31100,66,Откр.,22,20,"Свидетельствующий сие говорит: ей, гряду скоро..."


In [ ]:
# @title Для e5 нужна небольшая дополнетильная разметка.
df['Text']='passage: '+df['Text']
df

,BID,Book,Chapter,Verse,Text
0,1,Быт.,1,1,passage: В начале сотворил Бог небо и землю.
1,1,Быт.,1,2,"passage: Земля же была безвидна и пуста, и тьм..."
2,1,Быт.,1,3,passage: И сказал Бог: да будет свет. И сталсвет.
3,1,Быт.,1,4,"passage: И увидел Бог свет, что он хорош, и от..."
4,1,Быт.,1,5,"passage: И назвал Бог свет днем, а тьму ночью...."
...,...,...,...,...,...
31097,66,Откр.,22,17,passage: И Дух и невеста говорят: прииди! И сл...
31098,66,Откр.,22,18,passage: И я также свидетельствую всякому слыш...
31099,66,Откр.,22,19,passage: и если кто отнимет что от слов книги ...
31100,66,Откр.,22,20,"passage: Свидетельствующий сие говорит: ей, гр..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31102 entries, 0 to 31101
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   BID      31102 non-null  int64 
 1   Book     31102 non-null  object
 2   Chapter  31102 non-null  int64 
 3   Verse    31102 non-null  int64 
 4   Text     31102 non-null  object
dtypes: int64(3), object(2)
memory usage: 1.2+ MB


In [ ]:
s = df['Text'].tolist()

In [ ]:
# @title sbert_large_mt_nlu_ru
from transformers import AutoTokenizer, AutoModel
import torch
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask
#Sentences we want sentence embeddings for
#sentences = ['Привет! Как твои дела?',
#             'А правда, что 42 твое любимое число?']
#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
#Tokenize sentences
#encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt') # , max_length=24
#Compute token embeddings
#with torch.no_grad():
#    model_output = model(**encoded_input)
#Perform pooling. In this case, mean pooling
#sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
# @title multilingual-e5-large
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
# input_texts = ['query: how much protein should a female eat',
#                'query: 南瓜的家常做法',
#                "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#                "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Tokenize the input texts
# batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

# outputs = model(**batch_dict)
# embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
# embeddings = F.normalize(embeddings, p=2, dim=1)
# scores = (embeddings[:2] @ embeddings[2:].T) * 100
# print(scores.tolist())

In [ ]:
model.cuda()

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwi

In [ ]:
import torch
i = 0
sentence_embeddings = []
while i < len(s):
  #Tokenize sentences
  batch_dict = tokenizer(s[i:i+1000], max_length=512, padding=True, truncation=True, return_tensors='pt').to('cuda')
  #encoded_input = tokenizer(s[i:i+1000], padding=True, truncation=True, return_tensors='pt').to('cuda') # , max_length=24
  #Compute token embeddings
  with torch.no_grad():
      #model_output = model(**encoded_input)
      model_output = model(**batch_dict)
  #Perform pooling. In this case, mean pooling
  embeddings = average_pool(model_output.last_hidden_state, batch_dict['attention_mask'])
  embeddings = F.normalize(embeddings, p=2, dim=1)
  #print(model_output.last_hidden_state.shape, embeddings.shape)
  #sentence_embeddings.append(mean_pooling(model_output, encoded_input['attention_mask']))
  sentence_embeddings.append(embeddings) #.to('cpu'))
  i+=1000

In [ ]:
sentence_embeddings_ = torch.cat(sentence_embeddings)

In [ ]:
sentence_embeddings_ = sentence_embeddings_.to('cpu')

In [ ]:
sentence_embeddings_.shape

torch.Size([31102, 1024])

In [ ]:
from annoy import AnnoyIndex
import numpy as np

# Создание случайного массива эмбеддингов размера (num_embeddings, embedding_size)
num_embeddings = 31102
embedding_size = 1024
#embeddings = np.random.rand(num_embeddings, embedding_size).astype('float32')

# Инициализация индекса Annoy с использованием косинусной метрики
index = AnnoyIndex(embedding_size, metric='angular')


In [ ]:

# Добавление эмбеддингов в индекс
for i in range(num_embeddings):
    index.add_item(i, sentence_embeddings_[i])

# Построение индекса
index.build(10)

# Поиск ближайшего соседа для случайного запроса
query_embedding = np.random.rand(embedding_size).astype('float32')
indices = index.get_nns_by_vector(query_embedding, 5)

print("Indices:", indices)

Indices: [18046, 12246, 15514, 11555, 9888]


In [ ]:
index.save('rst.indx')

True

In [ ]:
index.load('rst.indx')

True

In [ ]:
df['Text']=df['Text'].str.replace('passage: ','')
df

,BID,Book,Chapter,Verse,Text
0,1,Быт.,1,1,В начале сотворил Бог небо и землю.
1,1,Быт.,1,2,"Земля же была безвидна и пуста, и тьма над без..."
2,1,Быт.,1,3,И сказал Бог: да будет свет. И сталсвет.
3,1,Быт.,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
4,1,Быт.,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
31097,66,Откр.,22,17,И Дух и невеста говорят: прииди! И слышавший д...
31098,66,Откр.,22,18,И я также свидетельствую всякому слышащему сло...
31099,66,Откр.,22,19,и если кто отнимет что от слов книги пророчест...
31100,66,Откр.,22,20,"Свидетельствующий сие говорит: ей, гряду скоро..."


In [ ]:
df.to_csv('rst_db.csv',index=False)

In [ ]:
sentence = ['query: Признаться ли ей в любви?']
#encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt').to('cuda') # , max_length=24
batch_dict = tokenizer(sentence, max_length=512, padding=True, truncation=True, return_tensors='pt').to('cuda')
  #Compute token embeddings
with torch.no_grad():
    model_output = model(**batch_dict)
  #Perform pooling. In this case, mean pooling
#sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
sentence_embeddings = average_pool(model_output.last_hidden_state, batch_dict['attention_mask'])
sentence_embeddings = F.normalize(embeddings, p=2, dim=1)


In [ ]:
sentence_embeddings[0]

tensor([-0.0003, -0.0314, -0.0329,  ...,  0.0013, -0.0403,  0.0077],
       device='cuda:0')

In [ ]:
indices = index.get_nns_by_vector(sentence_embeddings[0], 5)

In [ ]:
indices

[31000, 18607, 30763, 5667, 18606]

In [ ]:
import random

ind = random.choice(indices)
str_ = df.loc[ind].Text
f_ind = ind
while not df.loc[f_ind].Text.split()[0].istitle():
  f_ind-=1
  str_ = df.loc[f_ind].Text+' '+str_
l_ind = ind
while df.loc[l_ind].Text.find('.')<0:
  l_ind+=1
  str_ +=' '+df.loc[l_ind].Text

book = df.loc[f_ind]['Book']
chap = df.loc[f_ind]['Chapter']
if f_ind==l_ind:
  vers = df.loc[f_ind]['Verse']
else:
  vers = (str(df.loc[f_ind]['Verse'])+
          '-'+str(df.loc[l_ind]['Verse']))
str_+f' [{book}{chap}:{vers}]'


'Женщина жившая у тебя в неге и роскоши, которая никогда ноги своей не ставила на землю по причине роскоши и изнеженности, будет безжалостным оком смотреть на мужа недра своего и на сына своего и на дочьсвою и не даст им последа, выходящего из среды ног ее, и детей, которых она родит; потому что она, при недостатке во всем, тайно будет есть их, в осаде истеснении, в котором стеснит тебя враг твой в жилищах твоих. [Втор.28:56-57]'